# RESULTS
Here I report the resulsts of the training procedure for a single model. I just want to check if everything works as supposed to.

In [ ]:
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

%load_ext autoreload
%autoreload 2

In [ ]:
prms = {'L' : 10,                # length of spin chain
        'sites' : [0, 1],        # sites of the subsystem S spins
        'omega' : 1,             # Rabi frequency
        'beta' : 0.01,           # inverse temperature
        'potential' : 0.1,       # interaction of subsystem's S spins
        'potential_' : None,     # interaction of bath spins, if None same as potential
        'T' : 10,                # total time for the evolution
        'dt' : 0.01,             # interval every which save the data
        'cutoff' : 1e-8,         # cutoff for TEBD algorithm
        'im_cutoff' : 1e-10,      # cutoff for TEBD algorithm, img t-e
        'tolerance' : 1e-3,      # Trotter tolerance for TEBD algorithm
        'verbose' : True,        # verbosity of the script
        'num_traj' : 20,         # how many trajectories to do
        # file to save the data
        'fname' : './data/data_tebd.hdf5'
        }

ml_params = {'model_dir': './data/trained_model', # folder where the metadata of the training are stored
             'validation_split' : 0.8,
             'batch_size': 256,
             #'batches_per_epoch': 256,
             'n_epochs': 20,
             'device': 'cpu',
             'mlp_params': {
                 'data_dim': 15,
                 'layers': [],
                 'nonlin': 'id',
                 'output_nonlin': 'id',
                 'dt' : 0.01
                 },
             }

### Generate some data

In [ ]:
### TEBD
from models import SpinChain

sys_prms = prms.copy()
sys_prms.pop('sites')
sys_prms.pop('potential_')
sys_prms.pop('num_traj')
sys_prms.pop('fname')
sys_prms['verbose'] = False

# evolution of the spin chain
system = SpinChain(**sys_prms)
system.thermalize()
system.evolve(seed=33)

results = system.return_results()

In [ ]:
### EXACT
from models import SpinChain_ex

sys_prms = prms.copy()
sys_prms.pop('sites')
sys_prms.pop('potential_')
sys_prms.pop('num_traj')
sys_prms.pop('fname')
sys_prms['verbose'] = False
sys_prms.pop('cutoff')
sys_prms.pop('im_cutoff')
sys_prms.pop('tolerance')
sys_prms.pop('verbose')
# I simulate for longer times, I use it later
sys_prms['T'] = 20

# evolution of the spin chain
system_ex = SpinChain_ex(**sys_prms)
system_ex.evolve(seed=33)

results_ex = system_ex.return_results()

In [ ]:
t = np.arange(0, 10, prms['dt'])

rows= 5
columns = 3

fig, axs = plt.subplots(rows, columns, figsize=(15,15), dpi=80)
plt.setp(axs, xlim=(0,10), ylim=(-0.006, 0.006))

for i in range(rows):
    for j in range(columns):
        #if i == rows-1 & j == columns-1:
        #    continue
        axs[i, j].plot(t, [results_ex[k][(columns-1)*i+j] for k in range(len(t))], label='Exact', color='k', linewidth=2)
        axs[i, j].plot(t, [results[k][(columns-1)*i+j] for k in range(len(t))], label='TEBD', color='r')
        axs[i, j].grid()

#plt.legend()
plt.grid()

### Import the model

In [ ]:
from ml.classes import MLLP

name = 'model_L_' + str(prms['L']) + \
        '_V_' + str(int(prms['potential']*1e3)).zfill(4) + \
        '_beta_' + str(int(prms['beta']*1e3)).zfill(4) + \
        '_dt_' + str(int(prms['dt']*1e3)).zfill(4) + \
        '_T' + str(int(prms['T']))

my_file = Path('./data/trained_model/' + name)
if not my_file.is_file():
    raise Exception('File do not exists!')

model = MLLP(ml_params['mlp_params']).to(ml_params['device'])
model.load_state_dict(torch.load(my_file))

In [ ]:
for idx, m in enumerate(model.modules()):
    print(idx, '->', m)

In [ ]:
print(model)

### Generate the dynamics with the model

In [ ]:
ml_dyn = model.generate_trajectory(v_0=np.array(results[0]), T=10)

In [ ]:
from ml.utils import calculate_error

err = calculate_error(results_ml=ml_dyn, results_tebd=results, T=prms['T'], dt=prms['dt'])
print(err)

In [ ]:
t = np.arange(0, 10, prms['dt'])

rows= 5
columns = 3

fig, axs = plt.subplots(rows, columns, figsize=(15,15), dpi=80)
plt.setp(axs, xlim=(0,10), ylim=(-0.006, 0.006))

for i in range(rows):
    for j in range(columns):
        #if i == rows-1 & j == columns-1:
        #    continue
        axs[i, j].plot(t, [results[k][(columns-1)*i+j] for k in range(len(t))], label='Simulation', color='k')
        axs[i, j].plot(t, [ml_dyn[k][(columns-1)*i+j] for k in range(len(t))], label='ml', color='r', linestyle='--')
        axs[i, j].grid()
axs[4, 2].grid()
#plt.legend()
plt.grid()

fig = plt.gcf()
plt.show()

#fig.savefig('../pictures/first_model.pdf', dpi=80)

## Further times

In [ ]:
ml_dyn_long = model.generate_trajectory(v_0=np.array(results[0]), T=20)

In [ ]:
err = calculate_error(results_ml=ml_dyn_long, results_tebd=results_ex, T=20, dt=prms['dt'])
print(err)

In [ ]:
t = np.arange(0, 20, prms['dt'])

rows= 5
columns = 3

fig, axs = plt.subplots(rows, columns, figsize=(15,15), dpi=80)
plt.setp(axs, xlim=(0,20), ylim=(-0.006, 0.006))

for i in range(rows):
    for j in range(columns):
        #if i == rows-1 & j == columns-1:
        #    continue
        axs[i, j].plot(t, [results_ex[k][(columns-1)*i+j] for k in range(len(t))], label='Simulation', color='k')
        axs[i, j].plot(t, [ml_dyn_long[k][(columns-1)*i+j] for k in range(len(t))], label='ml', color='r', linestyle='--')
        axs[i, j].grid()
axs[4, 2].grid()
#plt.legend()
plt.grid()

fig = plt.gcf()
plt.show()

#fig.savefig('../pictures/first_model_further_times.pdf', dpi=80)